In [1]:
import subprocess, os, csv, pandas as pd
import wget, urllib
from Bio.SeqIO.FastaIO import SimpleFastaParser
from Bio.Seq import Seq
from Bio import SeqIO
from Bio import Entrez
import Bio
from Bio import UniGene

from Bio.Alphabet import generic_dna

$\text{Searching bacteriophages in virus list and hosts Tax_ID list}$

In [2]:
# Here I'm searching bacteriophages in the table (from Virus-Host DB FTP) and saving  result in list
# Also I saving host names in anpther list
virus_host_info = []
host_name = []
host_id = []
with open("/home/emil/Desktop/Science/virushostdb/virushostdb.tsv") as tsvfile:
    tsvreader = csv.reader(tsvfile, delimiter="\t",)
    for line in tsvreader:
        if ('Bacteria' in line[9]):
            virus_host_info.append(line)
            host_name.append(line[8])
            host_id.append(line[7])
        if (line[9] == 'host lineage'):
            virus_host_info.append(line)

$\text{Searchinh names and genomes of bacheriophages}$

$\text{First Method}$

In [3]:
#Here I'm searching bacteriophages genomes using an FTP from Virus-Host DB
f = open('virushostdb.formatted.genomic.fna', 'r')
s = f.read()
f.close()
lst1 = []
lst2 = []
Genome = s.split('>')
Genome.remove('')
lst = []
for i in range(len(Genome)):
    sf = Genome[i].split('|')
    lst.append(sf)
for i in range(0,len(lst)):
    sf1 = lst[i][0].split(' ')
    lst1.append(sf1)
for i in range(0,len(lst1)):
    s = ''
    for j in range(1,len(lst1[i])):
        s += lst1[i][j] + ' '
    index = len(s)-1
    s = s[:index] + s[index+1:]
    lst2.append(s) #Virus names
l = 0
GENOMES = []
NAMES = []
for i in range(1,len(virus_host_info)):
    p = lst2.index(virus_host_info[i][1])
    NAMES.append(lst2[p]) #Bacteriophage names
    GENOMES.append(lst[p][5]) #Bacteriophage genomes

$\text{Second Method (using biopython)}$

In [12]:
Entrez.email = "sharafutdinov.ehn@phystech.edu"
Genes = []
#Can I do it in loop?
handle = Entrez.efetch(db="nucleotide", id=virus_host_info[1][3], rettype="gb", retmode="text")
record = SeqIO.read(handle, "genbank")
handle.close()
Genes.append(repr(record))
f = open('OUTPUT.txt', 'w')
f.write(Genes[0])
f.close()
#print(Genes)

In [38]:
Entrez.email = "sharafutdinov.ehn@phystech.edu"# Always tell NCBI who you are
filename = "EU490707.gbk"
if not os.path.isfile(filename):# Downloading...
    net_handle = Entrez.efetch(db="nucleotide", id="EU490707", rettype="gb", retmode="text")
    out_handle = open(filename, "w")
    out_handle.write(net_handle.read())
    out_handle.close()
    net_handle.close()
    print("Saved")
    
print("Parsing...")
record = SeqIO.read(filename, "genbank")
record.seq

Parsing...


Seq('ATTTTTTACGAACCTGTGGAAATTTTTGGTTATGACAATAAATCTAGTTTAGTA...GAA', IUPACAmbiguousDNA())

$\text{Searching IDList from hosts names}$

In [21]:
#I know host names and here I'm searching their genomes
Entrez.email = "sharafutdinov.ehn@phystech.edu"
HOST_IDs = []
s = ''
j = 0
#Can I do it in loop?
s = host_name[1] + "[Orgn]"
handle = Entrez.esearch(db="nucleotide", term=s, idtype="acc")
record = Entrez.read(handle)
record["Count"]
HOST_IDs.append(record["IdList"])
#print((HOST_IDs))

$\text{I 'm looking for host genomes here}$

In [22]:
Entrez.email = "sharafutdinov.ehn@phystech.edu"
Genes_Host = []
#for i in range(len(HOST_IDs)): And can I do this in loop?
handle = Entrez.efetch(db="nucleotide", id=HOST_IDs[0][1], rettype="gb", retmode="text")
record = SeqIO.read(handle, "genbank")
handle.close()
Genes_Host.append(repr(record.seq))
#print((Genes_Host))

$\text{Searching hosts genome}$

In [10]:
data = pd.read_csv('/home/emil/Desktop/Science/assembly_summary_genbank.txt', sep="\n", header=None) #parsing
Assembly_summary_genbank = []
taxid = [] 
f = open('viruses_hosts_correlation_table', 'w')
f.write('viruses_id' + '\t' + 'hosts_id')
f.write('\n')
non_complete_genome_id = [] #Here IDs of organisms without full genome
link_to_sequence = [] #Here link to dowload data in FTP
for i in range(1,len(data[0])):
    s = data[0][i].split('\t')
    taxid.append(s[5])
    Assembly_summary_genbank.append(s)
for ID in host_id:
    if ID in taxid:
        index = taxid.index(ID)
        virus_index = host_id.index(ID) + 1
        f.write(virus_host_info[virus_index][3] + '\t' + Assembly_summary_genbank[index][0])
        f.write('\n')
        if (Assembly_summary_genbank[index][13] == 'Full' or Assembly_summary_genbank[index][11] == 'Chromosome'):
            link_to_sequence.append(Assembly_summary_genbank[index][19])
        else:
            non_complete_genome_id.append(ID)
    else:
        non_complete_genome_id.append(ID)
            
f.close()

$\text{Downloading Data}$

In [ ]:
unique_link = set(link_to_sequence)
unique_link = list(unique_link)
for i in range (len(link_to_sequence)):
    string = unique_link[i].split('/')
    link = unique_link[i] + '/' + string[len(string) - 1] + '_genomic.fna.gz'
    os.system('wget -r %s' %link)